## Franklin Reiter

Let $\langle e,N \rangle$ be the public key, and suppose $ m_1 = f(m_2) \mod N $, for some known $f \in \mathbb{Z_{N}}[x]$, where f is a linear polynomial ( $f(x) = ax + b $ ). Given $ c_1, c_2$, the algorithm can efficiently recover $m_1, m_2$ for any relatively small e.

Notice that $m_2$ is a root of both $ f(x)^e - c_1 \mod N $ and  $ x^e - c_2 \mod N $.
That said, we can apply polynomial G.C.D. in order to recover $m_2$.

The core idea is that for small exponents, the G.C.D is expected to be linear in most cases.

In [1]:
def bytes_to_long(b):
    return int(b.hex(), base=16)

def long_to_bytes(l):
    return bytes.fromhex(hex(l)[2:])

In [2]:
p = random_prime(2^1024)
q = random_prime(2^1024)

n = p * q

# 
e = 3

a = randint(0,2^16)
b = randint(0,2^16)

m_2 =  bytes_to_long(b"Well hidden message!!!! Lorem ipsum \
   dolor sit amet, consectetur adipiscing elit, \
   sed do eiusmod tempor incididunt ut labore ")

# m_2 = bytes_to_long(b"Well hidden message!!!!!")

m_1 = (a * m_2 + b) % n

c_2 = pow(m_2, e, n)
c_1 = pow(m_1, e, n)



The implementation below calculates the GCD in $\mathbb{Q}[x]$, thus works only when $x^{e}, f(x)^{e}$ are both less than $N$.

In [ ]:
from copy import copy


def polyDiv(x1, x2): 
    assert x2 != 0
    q = 0
    r, d = x1, x2
    # print(r.poly, d.poly)
    while r.poly != 0 and d.poly != 0 and r.degr() >= d.degr():
#         print(r.poly, r.lead(), d.lead())
        t = r.lead() / d.lead()
        
        
        
        q += t * xs ^ (r.degr() - d.degr())
        r.poly -= t * d.poly * xs ^ (r.degr() - d.degr())
        r.poly = r.poly.simplify_full()

#     print('polyDiv ', q, r)
    return Poly(q), r
    

def polyGCD(x1, x2):
    if x2.poly == 0:   
        return Poly(x1.poly / x1.lead())
    
    x1, x2 = x2, x1 % x2
#     print('polyGCD: ', x1, x2)
    
    
    return polyGCD(copy(x1), copy(x2))
    

class Poly:
    def __init__(self, poly):
        self.poly = poly
    
    def __repr__(self):
        return str(self.poly)
    
    def __eq__(self, other):
        if type(other) == type(self):
            return self.poly == other.poly
        else:
            return self.poly == other
        
    def __mod__(self, other):
        return polyDiv(self, other)[1]
    
    def degr(self):
        return self.poly.degree(xs)
    
    def lead(self):
        #print(self.poly.coefficient(xs, n=self.degr()), self.degr())
        return self.poly.coefficient(xs, n=self.degr())

    

    
xs = var('xs')
xx = Poly(xs ^ 3 + xs^2 + xs + 1)
xw = Poly(xs ^ 2 - 1)

res1 = polyGCD(copy(xx), copy(xw))

assert res1 == xs + 1


In [ ]:

m = var('xs')

P1 = (a*xs + b) ^ e - c_1
P2 = xs ^ e - c_2

P1 = Poly(P1)
P2 = Poly(P2)

print(P1, P2)
print(polyGCD(P1,P2))

msg = -polyGCD(P1, P2).poly.coefficient(xs, n=0)

print(msg)



We can edit this implementation so that it divides the polynomials in $ \mathbb{Z_N}[x] $

In [ ]:
###TODO
###add Zn solver from .sage file


def polyDivZn(x1, x2): 
    assert x2 != 0
    q = 0
    r, d = x1, x2
    # print(r.poly, d.poly)
    while r.poly != 0 and d.poly != 0 and r.degr() >= d.degr():
        print(type(d.lead()))
        d_i = Integer(d.lead()).inverse_mod(n)
        print(d_i)
#         print(r.poly, r.lead(), d.lead())
        t = (Integer(r.lead()) * d_i) % n
        
        
        
        q += t * xs ^ (r.degr() - d.degr())
        r.poly -= t * d.poly * xs ^ (r.degr() - d.degr())
        r.poly = r.poly.simplify_full()

#     print('polyDiv ', q, r)
    return Poly(q), r

def polyGCDZn(x1, x2):
    if x2.poly == 0:   
        return Poly(x1.poly * x1.inverse_mod(n))
    
    x1, x2 = x2, x1 % x2
    # print('polyGCD: ', x1, x2)
    
    
    return polyGCD(copy(x1), copy(x2))
    


class PolyZn:
    def __init__(self, poly):
        self.poly = poly
    
    def __repr__(self):
        return str(self.poly)
    
    def __eq__(self, other):
        if type(other) == type(self):
            return self.poly == other.poly
        else:
            return self.poly == other
        
    def __mod__(self, other):
        return polyDivZn(self, other)[1]
    
    def degr(self):
        return self.poly.degree(xs)
    
    def lead(self):
        #print(self.poly.coefficient(xs, n=self.degr()), self.degr())
        return self.poly.coefficient(xs, n=self.degr())

    
xs = var('xs')
xx = PolyZn(xs ^ 3 + xs^2 + xs + 1)
xw = PolyZn(xs ^ 2 - 1)

res1 = polyGCDZn(copy(xx), copy(xw))

assert res1 == xs + 1
    